In [1]:
import random
import numpy as np

Field parameters
=====

We shall be operating on fixed precision rational numbers. We shall use a finite prime field identified by `MODULUS` to represent these.

In [2]:
BASE = 10
PRECISION_INTEGRAL = 2
PRECISION_FRACTIONAL = 6
MODULUS = 1094402170831300646699956388099

PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL
INVERSE = 1092323901108892006771873170918 # inverse of BASE**FRACTIONAL_PRECISION

assert(MODULUS > BASE**(2*PRECISION))
assert((INVERSE * BASE**PRECISION_FRACTIONAL) % MODULUS == 1)

Fixed-Point Encoding
=====

We shall be using the fixed-point encoding scheme from [CS'10](https://www1.cs.fau.de/filepool/publications/octavian_securescm/secfp-fc10.pdf), with `e=PRECISION_INTEGRAL` and `f=PRECISION_FRACTIONAL`.

In [3]:
def encode(rational):
    assert(abs(rational) < BASE**PRECISION_INTEGRAL)
    upscaled = int(rational * BASE**PRECISION_FRACTIONAL)
    field_encoded = upscaled % MODULUS
    return field_encoded

def decode(field_encoded):
    signed = field_encoded if field_encoded < MODULUS/2 else field_encoded - MODULUS
    rational = signed / BASE**PRECISION_FRACTIONAL
    return rational

x = encode(0.123456789)
print("encoded: %s, decoded: %s" % (x, decode(x)))

y = encode(1/4)
print("encoded: %s, decoded: %s" % (y, decode(y)))

z = encode(-0.25)
print("encoded: %s, decoded: %s" % (z, decode(z)))

encoded: 123456, decoded: 0.123456
encoded: 250000, decoded: 0.25
encoded: 1094402170831300646699956138099, decoded: -0.25


Testing
-----
We are *not* going to use this in the actual computation, but it's useful to get an intuition about the encoding and the operations.

In [ ]:
def open_add(x, y):
    z = (x + y) % MODULUS
    return z

def open_sub(x, y):
    z = (x - y) % MODULUS
    return z

x = encode(0.12345)
y = encode(-0.00345)
z = open_add(x, y)
print("encoded: %s, decoded: %s" % (z, decode(z)))

x = encode(-0.12345)
y = encode(-0.00345)
z = open_sub(x, y)
print("encoded: %s, decoded: %s" % (z, decode(z)))

In [ ]:
def open_truncate(a):
    # map to the positive range
    b = (a + BASE**(2*PRECISION-1)) % MODULUS
    # extract lower digits
    b_low = b % BASE**PRECISION_FRACTIONAL
    # remove lower digits
    c = a - b_low
    # remove extra scaling factor
    d = (c * INVERSE) % MODULUS
    return d

def open_mul(x, y):
    # result with double precision
    w = (x * y) % MODULUS
    # result with single precision
    v = open_truncate(w)
    return v

x = encode(-.5)
y = encode(-.5)
print("encoded: %s, decoded: %s" % (x, decode(x)))

z = open_mul(x, y)
print("encoded: %s, decoded: %s" % (z, decode(z)))

v = open_mul(z, z)
print("encoded: %s, decoded: %s" % (v, decode(v)))

w = open_mul(v, v)
print("encoded: %s, decoded: %s" % (w, decode(w)))

Secure Arithmetic
=====

In [4]:
KAPPA = 5 # leave room for five digits overflow before leakage

assert(MODULUS > BASE**(2*PRECISION + KAPPA))

In [5]:
def share(secret):
    first  = random.randrange(MODULUS)
    second = random.randrange(MODULUS)
    third  = (secret - first - second) % MODULUS
    return [first, second, third]

def reconstruct(sharing):
    return sum(sharing) % MODULUS

secret = 5.0
encoded = encode(secret)
shared = share(encoded)
assert(decode(reconstruct(shared)) == secret)

print("shared: %s" % shared)

shared: [726594827352329060357659880166, 378751469946245956291264231604, 1083458044364026276750993664428]


In [6]:
def secure_add(x, y):
    # pair-wise addition (local computation)
    return [ (xi + yi) % MODULUS for xi, yi in zip(x, y) ]

def secure_sub(x, y):
    # pair-wise subtraction (local computation)
    return [ (xi - yi) % MODULUS for xi, yi in zip(x, y) ]

x = share(encode(.1))
y = share(encode(.5))
z = secure_add(x, y)
assert(decode(reconstruct(z)) == .1 + .5)

In [7]:
def secure_cmul(x, c):
    return [ (xi * c) % MODULUS for xi in x ]

def secure_truncate(a):
    # map to the positive range
    b = secure_add(a, [BASE**(2*PRECISION+1), 0, 0])
    # apply mask known only by P0, and reconstruct masked b to P1 or P2
    mask = random.randrange(MODULUS) % BASE**(PRECISION + PRECISION_FRACTIONAL + KAPPA)
    mask_low = mask % BASE**PRECISION_FRACTIONAL
    b_masked = reconstruct(secure_add(b, [mask, 0, 0]))
    # extract lower digits
    b_masked_low = b_masked % BASE**PRECISION_FRACTIONAL
    b_low = secure_sub(share(b_masked_low), share(mask_low))
    # remove lower digits
    c = secure_sub(a, b_low)
    # remove extra scaling factor
    d = secure_cmul(c, INVERSE)
    return d

def secure_mul(x, y):
    # local computation
    z0 = (x[0]*y[0] + x[0]*y[1] + x[1]*y[0]) % MODULUS
    z1 = (x[1]*y[1] + x[1]*y[2] + x[2]*y[1]) % MODULUS
    z2 = (x[2]*y[2] + x[2]*y[0] + x[0]*y[2]) % MODULUS
    # reshare and distribute (requires communication)
    Z = [ share(z0), share(z1), share(z2) ]
    w = [ sum(row) % MODULUS for row in zip(*Z) ] # transpose followed by row sum
    # truncate
    v = secure_truncate(w)
    return v

x = share(encode(-.5))
y = share(encode(-.00025))
z = secure_mul(x, y)
assert(decode(reconstruct(z)) == (-.5) * (-.00025))

In [8]:
def close(lhs, rhs):
    res = abs(lhs - rhs) <= 0.001
    if not res:
        print("close: %s" % lhs)
        print("close: %s" % rhs)
    return res

def inrange(val):
    res = abs(val) < BASE**PRECISION_INTEGRAL
    if not res:
        print("inrange: %s" % val)
    return res

class SecureRational(object):
    
    def __init__(self, secret=None):
        if secret is None:
            self.shares = []
        else:
            self.shares = share(encode(secret))
            assert(close(secret, self.reveal()))
            assert(inrange(self.reveal()))
    
    def reveal(self):
        return decode(reconstruct(self.shares))
    
    def __repr__(self):
        return "SecureRational(%f)" % self.reveal()
    
    def __add__(x, y):
        if not isinstance(y, SecureRational):
            y = SecureRational(y)
        z = SecureRational()
        z.shares = secure_add(x.shares, y.shares)
        assert(close(z.reveal(), x.reveal() + y.reveal()))
        assert(inrange(z.reveal()))
        return z
    
    def __sub__(x, y):
        if not isinstance(y, SecureRational):
            y = SecureRational(y)
        z = SecureRational()
        z.shares = secure_sub(x.shares, y.shares)
        assert(close(z.reveal(), x.reveal() - y.reveal()))
        assert(inrange(z.reveal()))
        return z
    
    def __mul__(x, y):
        if not isinstance(y, SecureRational):
            y = SecureRational(y)        
        z = SecureRational()
        z.shares = secure_mul(x.shares, y.shares)
        assert(close(z.reveal(), x.reveal() * y.reveal()))
        assert(inrange(z.reveal()))
        return z
    
    def __pow__(x, e):
        y = SecureRational(1)
        for _ in range(e):
            y = y * x
        assert(close(y.reveal(), x.reveal()**e))
        assert(inrange(y.reveal()))
        return y
    
x = SecureRational(.5)
y = SecureRational(-.25)
z = x * y
assert(z.reveal() == (.5) * (-.25))

print(z)

SecureRational(-0.125000)


In [9]:
# reseed to get reproducible results
random.seed(1)
np.random.seed(1)

# keep track of bounds of x
MAX = 0
MIN = 0
def update_maxmin(x):
    global MAX
    global MIN
    MAX = max(MAX, x.reveal())
    MIN = min(MIN, x.reveal())
    
# define Sigmoid approximation function
ZERO = SecureRational(0)
ONE = SecureRational(1)

def scalar_sigmoid(x):
    update_maxmin(x)
    return ZERO + (x**5 * .002083) - (x**3 * .020833) + (x * .25) + .5 

def scalar_sigmoid_deriv(x):
    return (ONE - x) * x

sigmoid = np.vectorize(scalar_sigmoid)
sigmoid_deriv = np.vectorize(scalar_sigmoid_deriv)

# helper function to go to secure space
secure = np.vectorize(lambda x: SecureRational(x))

# training inputs
X = secure(np.array([
            [0,0,1],
            [0,1,1],
            [1,0,1],
            [1,1,1]
        ]))

y = secure(np.array([[
            0,
            0,
            1,
            1
        ]]).T)

# initial weights 
synapse0 = secure(2 * np.random.random((3,1)) - 1)

# training
for i in range(10000):

    # forward propagation
    layer0 = X
    layer1 = sigmoid(np.dot(layer0, synapse0))
    
    # back propagation
    layer1_error = y - layer1
    layer1_delta = layer1_error * sigmoid_deriv(layer1)
    
    # update
    update = np.dot(layer0.T, layer1_delta)
    synapse0 += update

# result
print(layer1)
print(MIN)
print(MAX)

[[SecureRational(0.000665)]
 [SecureRational(0.000441)]
 [SecureRational(0.999693)]
 [SecureRational(0.999469)]]
-2.487344
2.487863


In [ ]:
random.seed(1)
np.random.seed(1)

ZERO = SecureRational(0)
ONE = SecureRational(1)

def scalar_sigmoid(x):
    return ZERO + (x**5 * .002083) - (x**3 * .020833) + (x * .25) + .5 

def scalar_sigmoid_deriv(x):
    return (ONE - x) * x

sigmoid = np.vectorize(scalar_sigmoid)
sigmoid_deriv = np.vectorize(scalar_sigmoid_deriv)
secure = np.vectorize(lambda x: SecureRational(x))


X = secure(np.array([
            [0,0,1],
            [0,1,1],
            [1,0,1],
            [1,1,1]
        ]))

y = secure(np.array([[
            0,
            0,
            1,
            1
        ]]).T)

synapse0 = secure(2 * np.random.random((3,4)) - 1)
synapse1 = secure(2 * np.random.random((4,1)) - 1)

for i in range(10000):
    
    # forward propagation
    layer0 = X
    layer1 = sigmoid(np.dot(layer0, synapse0))
    layer2 = sigmoid(np.dot(layer1, synapse1))
    
    if i % 500 == 0:
        print(i)
    
    # back propagation
    layer2_error = y - layer2
    layer2_delta = layer2_error * sigmoid_deriv(layer2)
    layer1_error = np.dot(layer2_delta, synapse1.T)
    layer1_delta = layer1_error * sigmoid_deriv(layer1)
    
    # update
    synapse1 += np.dot(layer1.T, layer2_delta)
    synapse0 += np.dot(layer0.T, layer1_delta)
    
print(layer2)

Inlined Mul
======

In [ ]:
def share2(secret):
    first  = random.randrange(MODULUS)
    second = (secret - first) % MODULUS
    return [first, second]

def secure_mul(x, y):
    # local computation
    v0 = (x[0]*y[0] + x[0]*y[1] + x[1]*y[0]) % MODULUS
    v1 = (x[1]*y[1] + x[1]*y[2] + x[2]*y[1]) % MODULUS
    v2 = (x[2]*y[2] + x[2]*y[0] + x[0]*y[2]) % MODULUS
    # reshare and distribute (requires communication)
    B = [ share2(v0), share2(v1), share2(v2) ]
    c = [ sum(row) % MODULUS for row in zip(*B) ] # transpose followed by row sum
    # generate and distribute mask (requires communication)
    mask = random.randrange(MODULUS) // BASE**KAPPA
    mask_top = mask // BASE**PRECISION
    m = share3(mask_top)
    # reconstruct and truncate masked value, then reshare and distribute (requires communication)
    d = [ c[0] + mask, c[1] ]
    e = share3(reconstruct(d) // BASE**PRECISION)
    # remove mask
    z = [ (ei - mi) % MODULUS for ei, mi in zip(e, m) ]
    return z